In [2]:
import pandas as pd

# Define file paths (update the folder name if needed)
base_path = '/content/drive/MyDrive/travel dataset/'

destinations_path = base_path + 'destinations.csv'
weather_path = base_path + 'weather.csv'
events_path = base_path + 'events.csv'
reviews_path = base_path + 'reviews.csv'
social_media_path = base_path + 'social_media.csv'
flights_path = base_path + 'flights.csv'

# Load CSV files into Pandas DataFrames
destinations = pd.read_csv(destinations_path)
weather = pd.read_csv(weather_path)
events = pd.read_csv(events_path)
reviews = pd.read_csv(reviews_path)
social_media = pd.read_csv(social_media_path)
flights = pd.read_csv(flights_path)

# Check the first few rows of one DataFrame
print("Destinations:")
print(destinations.head())


Destinations:
   DestinationID      City    Country   Region  \
0              1     Paris     France   Europe   
1              2     Tokyo      Japan     Asia   
2              3  New York        USA  America   
3              4    London         UK   Europe   
4              5    Sydney  Australia  Oceania   

                                 Description  
0  City of Lights, known for art and cuisine  
1   Bustling metropolis with rich traditions  
2        The Big Apple with iconic landmarks  
3         Historic city with modern vibrancy  
4   Harbour city with the iconic Opera House  


In [10]:
# Step 3: Create an SQLite database and import the data
import sqlite3

# Create a connection to an in-memory SQLite database.
conn = sqlite3.connect(':memory:')

# Write each DataFrame to an SQLite table.
destinations.to_sql('destinations', conn, index=False, if_exists='replace')
weather.to_sql('weather', conn, index=False, if_exists='replace')
events.to_sql('events', conn, index=False, if_exists='replace')
reviews.to_sql('reviews', conn, index=False, if_exists='replace')
social_media.to_sql('social_media', conn, index=False, if_exists='replace')
flights.to_sql('flights', conn, index=False, if_exists='replace')

print("Data successfully imported into SQLite.")


Data successfully imported into SQLite.


In [11]:
# Step 4: Run SQL queries and analyze the data

# Query 1: Join destinations and weather data to show weather details by city.
query1 = """
SELECT d.City, d.Country, w.Date, w.Temperature, w.Condition
FROM destinations d
JOIN weather w ON d.DestinationID = w.DestinationID
ORDER BY d.City;
"""
weather_details = pd.read_sql_query(query1, conn)
print("Weather Details by City:")
print(weather_details)

# Query 2: Retrieve upcoming events with destination details.
query2 = """
SELECT d.City, e.EventName, e.EventDate, e.EventCategory
FROM destinations d
JOIN events e ON d.DestinationID = e.DestinationID
ORDER BY e.EventDate;
"""
upcoming_events = pd.read_sql_query(query2, conn)
print("\nUpcoming Events:")
print(upcoming_events)

# Query 3: Calculate average review ratings for each destination.
query3 = """
SELECT d.City, AVG(r.Rating) AS AvgRating
FROM destinations d
JOIN reviews r ON d.DestinationID = r.DestinationID
GROUP BY d.City
ORDER BY AvgRating DESC;
"""
avg_ratings = pd.read_sql_query(query3, conn)
print("\nAverage Review Ratings by City:")
print(avg_ratings)

# Query 4: List flight details for each destination.
query4 = """
SELECT d.City, f.Origin, f.Price, f.Duration, f.DepartureDate
FROM destinations d
JOIN flights f ON d.DestinationID = f.DestinationID
ORDER BY f.DepartureDate;
"""
flight_details = pd.read_sql_query(query4, conn)
print("\nFlight Details:")
print(flight_details)


Weather Details by City:
             City       Country        Date  Temperature Condition
0         Beijing         China  2025-04-01         10.0     Rainy
1          Berlin       Germany  2025-04-01         14.0  Overcast
2       Cape Town  South Africa  2025-04-01         17.0     Windy
3          London            UK  2025-04-01         12.0     Rainy
4        New York           USA  2025-04-01         20.0     Sunny
5           Paris        France  2025-04-01         16.0    Cloudy
6  Rio de Janeiro        Brazil  2025-04-01         25.0     Sunny
7            Rome         Italy  2025-04-01         15.0    Cloudy
8          Sydney     Australia  2025-04-01         22.0     Sunny
9           Tokyo         Japan  2025-04-01         18.0     Sunny

Upcoming Events:
             City                EventName   EventDate  EventCategory
0  Rio de Janeiro             Rio Carnival  2025-02-25       Cultural
1           Tokyo          Tokyo Tech Fair  2025-04-15           Tech
2         

In [12]:
# Step 5: Visualize the results using Plotly (or any other visualization library)

import plotly.express as px

# Visualize weather details: Temperature Trend by City
fig_weather = px.bar(weather_details, x='City', y='Temperature',
                       title="Average Temperature by City on " + weather_details['Date'].iloc[0],
                       labels={"Temperature": "Temperature (°C)"})
fig_weather.show()

# Visualize upcoming events: Number of Events per Event Category
event_counts = upcoming_events['EventCategory'].value_counts().reset_index()
event_counts.columns = ['EventCategory', 'Count']
fig_events = px.pie(event_counts, names='EventCategory', values='Count',
                    title="Event Categories Distribution")
fig_events.show()

# Visualize average review ratings by City
fig_reviews = px.bar(avg_ratings, x='City', y='AvgRating',
                     title="Average Review Ratings by City",
                     labels={"AvgRating": "Average Rating"})
fig_reviews.show()

# Visualize flight prices by Departure Date
fig_flights = px.scatter(flight_details, x='DepartureDate', y='Price',
                         color='City', size='Duration',
                         title="Flight Prices vs. Departure Dates",
                         labels={"Price": "Price (USD)"})
fig_flights.show()


In [13]:
# Step 6: Close the SQLite connection
conn.close()
print("SQLite connection closed.")


SQLite connection closed.
